# sched
- The sched module defines a class which implements a general purpose event scheduler

### Import package

In [1]:
import sched
import time

In [2]:
# function that will be scheduled
def func(*arg):
    print("Arguments are {}.".format(arg))
    print("Current time {}".format(time.time()))

## 1. Schedule task to happen after 3 seconds - with `enterabs()`

In [3]:
print("Current time {}".format(time.time()))
# default argument for sched.scheduler(): timefunc=time.monotonic, delayfunc=time.sleep
# Create scheduler
scheduler = sched.scheduler(timefunc=time.time, delayfunc=time.sleep)  
# Function will start in 3 seconds
start_in = 3  
# schedule task
scheduler.enterabs(time=time.time() + start_in, priority=1, action=func, argument=(1,2,3))  
# Run scheduler
scheduler.run()  

Current time 1560584363.4414299
Arguments are (1, 2, 3).
Current time 1560584366.4483929


## 2. Schedule task to happen after 3 seconds - with `enter()`

In [4]:
print("Current time {}".format(time.time()))
scheduler = sched.scheduler(timefunc=time.time, delayfunc=time.sleep)  
# Function will start in 3 seconds
delay = 3  
scheduler.enter(delay=delay, priority=1, action=func, argument=(1,2,3))  
scheduler.run()  

Current time 1560584366.476244
Arguments are (1, 2, 3).
Current time 1560584369.480575


## 3. Cancel scheduled task & check schedule queue
- `scheduler.empty()`
- `scheduler.cancel(event)`
- `scheduler.queue`

In [5]:
scheduler = sched.scheduler(timefunc=time.time, delayfunc=time.sleep)  
delay = 3 
# Schedule 1st task
task_1 = scheduler.enter(delay=delay, priority=1, action=func, argument=(1,2,3))
print("# of tasks in scheduler queue: {}".format(len(scheduler.queue)))
print("Is queue empty? {}\n".format(scheduler.empty()))
scheduler.run()  
print("\nIs queue empty? {}".format(scheduler.empty()))

# of tasks in scheduler queue: 1
Is queue empty? False

Arguments are (1, 2, 3).
Current time 1560584372.52094

Is queue empty? True


In [6]:
scheduler = sched.scheduler(timefunc=time.time, delayfunc=time.sleep)  
delay = 3 
# Schedule 2 tasks
task_1 = scheduler.enter(delay=delay, priority=1, action=func, argument=(1,2,3))
task_2 = scheduler.enter(delay=delay, priority=1, action=func, argument=(1,2,3))
print("# of tasks in before cancel task: {}".format(len(scheduler.queue)))
scheduler.cancel(task_2)
print("# of tasks in after cancel task: {}".format(len(scheduler.queue)))
scheduler.run()  


# of tasks in before cancel task: 2
# of tasks in after cancel task: 1
Arguments are (1, 2, 3).
Current time 1560584375.5385768


## 4. Periodic task with sched
#### Modify function

In [7]:
def func(scheduler, delay, *arg):
    print("Arguments are {}.".format(arg))
    print("Current time {}".format(time.time()))
    scheduler.enter(delay, 1, func, argument=(scheduler, delay, *arg))

#### Schedule as period task (interrupt notebook to stop)

In [ ]:
delay = 3
scheduler = sched.scheduler(timefunc=time.time, delayfunc=time.sleep)  
scheduler.enter(delay, 1, func, (scheduler, delay, 1,2,3))
scheduler.run()

## 5. Periodic task using decorator
#### Set delay & scheduler

In [8]:
from functools import wraps

delay = 3
scheduler = sched.scheduler(timefunc=time.time, delayfunc=time.sleep)  

#### Create decorator: use decorator to avoid repeat code & keep original function interface

In [9]:
def schedule_task(scheduler, delay, priority):
    def actual(func):
        @wraps(func)
        def decorate(*arg):
            scheduler.enter(delay, priority, decorate, argument=arg)
            func(*arg)
        return decorate
    return actual

#### Decorating function

In [10]:
@schedule_task(scheduler, delay, priority=1)
def func(*arg):
    print("Arguments are {}.".format(arg))
    print("Current time {}".format(time.time()))

#### Run scheduler

In [ ]:
scheduler.enter(delay, priority=1, action=func, argument=(1,2,3))
scheduler.run()